# Crisis Mapping Toolkit Documentation

This document provides a high level overview of how to use the Crisis Mapping Toolkit (CMT).  The CMT is a set of tools built using Google's Earth Engine Python API so familiariaty with that API will be extremely useful when working with the CMT.

# Installing Earth Engine

See instructions from Google here: https://docs.google.com/document/d/1tvkSGb-49YlSqW3AGknr7T_xoRB1KngCD3f2uiwOS3Q/edit

# "Hello Crisis Mapping Toolkit"

### Initialize Earth Engine

In [1]:
import sys
import os
import ee

# This script assumes your authentification credentials are stored as operatoring system
# environment variables.
__MY_SERVICE_ACCOUNT = os.environ.get('MY_SERVICE_ACCOUNT')
__MY_PRIVATE_KEY_FILE = os.environ.get('MY_PRIVATE_KEY_FILE')

# Initialize the Earth Engine object, using your authentication credentials.
ee.Initialize()

### Load the Crisis Mapping Toolkit

In [2]:
# Make sure that Python can find the CMT source files
CMT_INSTALL_FOLDER = '/home/smcmich1/repo/earthEngine/CrisisMappingToolkit/'
sys.path.append(CMT_INSTALL_FOLDER)
import cmt.util.evaluation
from cmt.mapclient_qt import centerMap, addToMap

### Load a domain

A domain is a geographic location associated with certain sensor images, global data sets, and other supporting files.  A domain is described by a custom XML file and can easily be loaded in Python.  Once the XML file is loaded all of the associated data can be easily accessed.  Note that none of the images are stored locally; instead they have been uploaded to web storage locations where Earth Engine can access them.

In [3]:
import cmt.domain
domainPath = os.path.join(CMT_INSTALL_FOLDER, 'config/domains/modis/kashmore_2010_8.xml')
kashmore_domain = cmt.domain.Domain(domainPath)

### Display the domain

In [4]:
import cmt.util.gui_util
cmt.util.gui_util.visualizeDomain(kashmore_domain)

A GUI should appear in a separate window displaying the domain location.  If the GUI does not appear, try restarting the IPython kernel and trying again.  This is the default GUI used by the CMT.  It is an enhanced version of the GUI provided with the Earth Engine Python API and behaves similarly to the Earth Engine online "playground" interface.

#### Basic GUI instructions:
- You can move the view location by clicking and dragging.
- You can zoom in and out using the mouse wheel.
- Right clicking the view brings up a context menu with the following:
  - The lat/lon coordinate where you clicked.
  - The list of currently loaded image layers.
  - An opacity slider for each image layer.
  - The value for each image layer at the location you clicked.
  - A button which will save the current view as a geotiff file.
  
### Call a classification algorithm

In [5]:
from cmt.modis.flood_algorithms import *

# Select the algorithm to use and then call it
algorithm = DIFFERENCE
(alg, result) = detect_flood(kashmore_domain, algorithm)

# Get a color pre-associated with the algorithm, then draw it on the map
color = get_algorithm_color(algorithm)
addToMap(result.mask(result), {'min': 0, 'max': 1, 'opacity': 0.5, 'palette': '000000, ' + color}, alg, False)

#### Classifier output
- The algorithm output should have been added to the GUI as another image layer.
- Each classifier algorithm evaluates each pixel as flooded(1) or dry (0).  Some algorithms will return a probability of being flooded ranging from 0 to 1.

### Evaluate classification results

In [6]:
precision, recall, eval_count, quality = cmt.util.evaluation.evaluate_approach(result, kashmore_domain.ground_truth, kashmore_domain.bounds, is_algorithm_fractional(algorithm))
print('For algorithm "%s", precision = %f and recall = %f' % (alg, precision, recall) )
    

For algorithm "Difference", precision = 0.958887 and recall = 0.942759


#### Interpreting results

The two main scores for evaluating an algorithm are "precision" and "recall".  
- Precision is a measure of how many false positives the algorithm has.  It is calculated as: (number of pixels classified as flooded which are actually flooded) / (number of pixels classified as flooded)
- Recall is a measure of how sensitive to flooding the algorithm is.  It is calculated as: (number of pixels classified as flooded which are actually flooded) / (total number of flooded pixels)

In order for these measurements to be computed the domain must have a ground truth file associated with it which labels each pixel as flooded or dry.


#### End of introduction

The documentation so far covers most of the code used to write a file such as the tool detect_flood_modis.py.  The rest of the documentation covers different aspects of the CMT in more detail.


# Supported Sensor Data

The Crisis Mapping Toolkit has so far been used with the following types of data:
- MODIS = 250m to 500m satellite imagery covering the globe daily.
- LANDSAT = 30m satellite imagery with global coverage but infrequent images.
- DEM = Earth Engine provides the SRTM90 and NED13 digital elevation maps.
- Skybox = Google owned RGBN imaging satellites.
- SAR = Cloud penetrating radar data.  Several specific sources have been tested:
  - UAVSAR
  - Sentinel-1
  - Terrasar-X

MODIS and LANDSAT data are the easiest types to work with because Earth Engine already has all of that data loaded and easily accessible.  SAR data on the other hand can be difficult or expensive to get ahold of.

Most of the processing algorithms currently in CMT are for processing MODIS or SAR data and are split between the modis and radar folders.  Some of the algorithms, such as the active contour, can also operate on other types of data.

Instructions for how to load your own data are located in the "Domains" section of this documentation.



# Algorithm Overviews

The algorithms currently implemented by the CMT fall into these categories:

MODIS
 - Simple algorithms = Basic thresholding and small decision tree algorithms.
 - EE Classifiers = These algorithms are built around Earth Engine's classifier tool.
 - DNNS = Variants of the DNNS algorithm (http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=6307841)
 - Adaboost = Uses multiple instances of the simple algorithms to build a more accurate composite classification.
 - Misc algorithms = A few other algorithms outside those categories.

RADAR
 - Learning = Algorithms built around Earth Engine's classifier tool.
 - Matgen = An algorithm which attempts to detect water using find a global histogram split. (http://www.sciencedirect.com/science/article/pii/S1474706510002160)
 - Martinis = Breaks up the region into sub-regions to try and obtain a more useful histogram to split (http://www.nat-hazards-earth-syst-sci.net/9/303/2009/nhess-9-303-2009.pdf)
 - Active Contour = A "snake" algorithm for finding water boundaries.

Skybox
 - The MODIS EE Classifiers can incorporate Skybox imagery to improve their results.
 - The Active Contour algorithm can be used on Skybox data.



# The Production GUI

In addition to the default GUI, the Crisis Mapping Toolkit has another GUI customized to perform a few useful operations.  It is accessible by running the "flood_detection_wizard.py" tool.  The main map portion of the production GUI is the same as in the default GUI but there are additional controls above and below the map window.

#### Why use the production GUI?
- Easily search through MODIS and Landsat data.  The production GUI lets you quickly change the date and then searches for the closest Landsat data.
- Quickly perform basic MODIS flood detection.  The controls at the bottom allow quick tuning of a simple flood detection algorithm on the currently displayed MODIS data.
- Generate training data.  You can use the production GUI to create labeled training polygons to load into several of the classifier algorithms.

<br>
<img src="production_gui_screenshot.png">
<center> A screenshot of the Production GUI </center>

#### Top buttons from left to right
- Date Selector Button = Choose the date of interest.  MODIS data will be loaded from that date and LANDSAT data will be searched for nearby that date.
- Set Processing Region = When clicked the current field of view in the map will be set as the region of interest.  This region is used when searching for LANDSAT images and performing flood detection.
- Load Images = Once the data and region have been set, press this button to search for MODIS and LANDSAT data.  The data should be added to the main map display.
- Detect Flood = Run a flood detection algorithm using the values currently set by the sliders at the bottom of the GUI.  Flood detection results will be displayed in the main map display.
- Load Maps Engine Image = Paste the full Earth Engine ID from an image loaded in Google Maps Engine, then select the associated sensor type and click "Ok".  The image will now be displayed on the main map display.  Currently only one image at a time is supported.
- Open Class Trainer = Opens another window for generating training regions.
- Clear Map Button = Click this to remove all images from the main map display.

#### How to load MODIS/LANDSAT data
- Click the date select button and pick a date.
- Pan and zoom to your region of interest and click "Set Processing Region".
- Click "Load Images"

#### How to detect floods
- Perform the three steps above to load MODIS and LANDSAT data.
- Adjust the two sliders at the bottom to set the algorithm parameters.
    - Change Detection Threshold = Decrease this value to detect more pixels as flooded.
    - Water Mask Threshold = Increase this value to detect more pixels as flooded.
- Click "Detect Flood"

#### How to generate training regions for classifiers
- Load the imagery you want to look at while selecting regions, either MODIS/LANDSAT data or by  clicking "Load ME image".
- Click "Open Class Trainer"
- Use the text editor box to enter the name of a region.  Each name should contain either "Land" or "Water" to let the classifiers know how to use that region.
- Press "Add New Class" to add the named region to the class list.
- To select a class, click its name in the list.  When a class is selected you cannot drag the map view around!
- To unselect a class (so you can reposition the map) click "Deselect Class"
- You can delete a selected class from the list by clicking "Delete Class"
- To set the region for a selected class just click on locations in the main map view.  The points you click will form a polygon which should be drawn in the main map view.
- The main map view should keep updated with the polygon of the currently selected class but you may see some transient drawing artifacts.
- Click "Save Class File" to write a json file storing the training data.
- Click "Load Class File" to load an existing json class file.








# Working With Domains

#### The Domain Concept

A Domain consists of a region, training information, and a list of descriptions of available sensor data.  They can be easily loaded from XML files and the existing algorithms are all designed to take domain objects as input.  MODIS and DEM data are almost always available in any domain.  Instructions for creating a custom domain XML file are in the next section.

#### Anatomy of a Domain File

To use a custom domain generally requires three files:
- A sensor definition XML file.  Only one of these is needed per sensor.  It defines the bands, data characteristics, and possibly the data source.
- A test domain XML file.  This defines the geographic region, algorithm parameters, training and truth information, dates, and other other source information.
- A training domain XML file.  This is similar to the test domain file except that it will specify a different date or location to collect training data from.


For more detailed descriptions of all the possible contents of a domain file, check out the domain_example and sensor_example XML files and all of the real config files that are included with the Crisis Mapping Toolkit.

#### Code Examples

Here are some examples of code working with the Domain class in Python:


In [ ]:
# Access a specific parameter listed in the domain file
kashmore_domain.algorithm_parameters['modis_diff_threshold']

# Call this function to get whatever digital elevation map is available.
dem = kashmore_domain.get_dem()

# All the sensors included in the domain are stored as a list
first_sensor = kashmore_domain.sensor_list[0]

# If you know the name of a sensor you can access it like this
modis_sensor = kashmore_domain.modis

# Then you can access individual sensor bands like this
one_band = modis_sensor.sur_refl_b03

# To get the EE image object containing all the bands, do this
all_bands = modis.image

# The sensor contains some other information,
#   but only if the information is present in the XML files
first_band_name       = band_names[0]
first_band_resolution = modis.band_resolutions[first_band_name]

# Related domains have the same structure as the main domain
#  and can be accessed like this
kashmore_domain.training_domain
kashmore_domain.unflooded_domain


# Other Crisis Mapping Features

#### The Local Image Class
Earth Engine is very powerful but it is not well suited for all tasks.  In these cases you can use the LocalEEImage class to easily download image data from Earth Engine and work with it locally using whatever Python image processing method you prefer.  You can see an example of doing this in the Active Contour algorithm.

